In [18]:
import numpy as np
import pandas as pd

In [1]:
import mlflow
mlflow.set_tracking_uri(uri="http://localhost:5000")

In [8]:
import lightgbm as lgb

lgbc = lgb.Booster(model_file='lightgbm/lgb.txt')

We used our saved lightgbm model 

In [24]:
np.argmax(lgbc.predict([[-0.657196,-2.27162,1.324874,-0.097875,3.637970,-3.413761,0.790723217]]),axis=1).item()

1

In [31]:
mlflow.set_experiment('2_model_registry')
with mlflow.start_run():
    mlflow.log_params(lgbc.params)
    mlflow.set_tag("Model","lightgbm")

    mlflow.sklearn.log_model('lgbc',artifact_path='models')
    mlflow.set_tag("Prediction",np.argmax(lgbc.predict([[-0.657196,-2.27162,1.324874,-0.097875,3.637970,-3.413761,0.790723217]]),axis=1).item())

2024/01/31 13:51:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


/home/ubuntu/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ubuntu/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
